In [1]:
!pip install imblearn

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline
import joblib
from joblib import load

In [3]:

# %cd ..
#%cd MADS/Capstone/MADS-Capstone
%cd data
%pwd

/Users/hannah/Documents/capstone/data


'/Users/hannah/Documents/capstone/data'

In [4]:
df = pd.read_csv("cleaned_data.csv")

# take random sample of dataframe to reduce size
df = df.sample(n=200000, random_state=1)

# Drop discharge variable which was not dropped during preprocessing
df = df.drop(columns = "YEAR_OF_DISCHARGE")

#df.head()

In [5]:
print(df.columns)
print(df.shape)

Index(['AGE', 'GENDER', 'RACE', 'MARITAL_STATUS', 'EDUCATION',
       'EMPLOYMENT_AT_ADMISSION', 'LIVING_ARRANGEMENT_AT_ADMISSION',
       'ARRESTS_IN_30_DAYS_PRIOR_TO_ADMISSION', 'SERVICES_AT_ADMISSION',
       'REASON_FOR_DISCHARGE', 'PRIMARY_SOURCE_OF_REFERRAL',
       'PRIOR_TREATMENT_EPISODES', 'PRIMARY_SUBSTANCE_ABUSE',
       'FREQUENCY_OF_USE', 'AGE_AT_FIRST_USE', 'ALCOHOL_OR_DRUG_ABUSE',
       'DSM_DIAGNOSIS', 'PSYCHIATRIC_PROBLEM', 'HEALTH_INSURANCE',
       'PRIMARY_PAYMENT_METHOD', 'FREQUENCY_OF_SELF_HELP_ATTENDANCE', 'STATE'],
      dtype='object')
(200000, 22)


## Feature Engineering

In [6]:
print(df['REASON_FOR_DISCHARGE'].value_counts(), '\n')
print(df['SERVICES_AT_ADMISSION'].value_counts(), '\n')
print(df['PRIOR_TREATMENT_EPISODES'].value_counts())

Treatment completed                                     83860
Dropped out of treatment                                49786
Transferred to another treatment program or facility    42676
Terminated by facility                                  11450
Other                                                    8569
Incarcerated                                             3178
Death                                                     481
Name: REASON_FOR_DISCHARGE, dtype: int64 

Ambulatory, non-intensive outpatient                100515
Detox, 24-hour, free-standing residential            29734
Ambulatory, intensive outpatient                     26943
Rehab/residential, short term (30 days or fewer)     20782
Rehab/residential, long term (more than 30 days)     15127
Detox, 24-hour, hospital inpatient                    4931
Ambulatory, detoxification                            1500
Rehab/residential, hospital (non-detox)                468
Name: SERVICES_AT_ADMISSION, dtype: int64 

One or 

In [7]:
df = df[df['REASON_FOR_DISCHARGE'] != 'Transferred to another treatment program or facility']

In [8]:
# Create target variable. If the patient completed treatment and had no prior treatment episodes, they are considered a success. Otherwise, they are considered a failure.
df['SUCCESSFUL_TREATMENT'] = df.apply(lambda row: 1 if row['REASON_FOR_DISCHARGE'] == 'Treatment completed' and row['PRIOR_TREATMENT_EPISODES'] == "No prior treatment episode" else 0, axis=1)

print(df['SUCCESSFUL_TREATMENT'].value_counts())

0    129364
1     27960
Name: SUCCESSFUL_TREATMENT, dtype: int64


## Modeling

### Initial evaluation/model baselines

In [9]:
df_one_hot = pd.get_dummies(df)
print(df_one_hot.shape)

#for col in df_one_hot.columns:
#   print(col)

(157324, 203)


In [10]:
target = df['SUCCESSFUL_TREATMENT']
features = df.drop(['REASON_FOR_DISCHARGE', 'PRIOR_TREATMENT_EPISODES', 'SUCCESSFUL_TREATMENT'], axis=1)
features_one_hot = pd.get_dummies(features)

X_train, X_test, y_train, y_test = train_test_split(features_one_hot, target, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(125859, 193) (31465, 193) (125859,) (31465,)


In [11]:
X_train

,AGE_12-14,AGE_15-17,AGE_18-20,AGE_21-24,AGE_25-29,AGE_30-34,AGE_35-39,AGE_40-44,AGE_45-49,AGE_50-54,...,STATE_South Carolina,STATE_South Dakota,STATE_Tennessee,STATE_Texas,STATE_Utah,STATE_Vermont,STATE_Virginia,STATE_Washington,STATE_Wisconsin,STATE_Wyoming
3959568,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2063403,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1273168,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1396893,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1507988,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4548177,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5558166,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4581465,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3623378,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# for col in X_train.columns:
#     print(col)

In [13]:
# models_to_train = {
#     "naive_bayes": GaussianNB(),
#     "logistic_regression": LogisticRegression(max_iter=1000),
#     "random_forest": RandomForestClassifier()
# }

# for model_name, model in models_to_train.items():
#     model.fit(X_train, y_train)
#     y_pred = model.predict(X_test)
#     print(f"Accuracy score for {model_name}: {accuracy_score(y_test, y_pred)}")
#     print(f"f1 score for {model_name}: {f1_score(y_test, y_pred)}", '\n')

## Balance the data 

In [14]:
ros = RandomOverSampler(random_state=42)
X_train_res, y_train_res = ros.fit_resample(X_train, y_train)

print(y_train_res.value_counts())

0    103511
1    103511
Name: SUCCESSFUL_TREATMENT, dtype: int64


In [15]:
print(X_train_res.shape, y_train_res.shape)

(207022, 193) (207022,)


In [16]:
for col in X_train_res.columns:
    print(col)

AGE_12-14
AGE_15-17
AGE_18-20
AGE_21-24
AGE_25-29
AGE_30-34
AGE_35-39
AGE_40-44
AGE_45-49
AGE_50-54
AGE_55-64
AGE_65+
GENDER_Female
GENDER_Male
GENDER_Not known
RACE_Alaskan Native
RACE_American Indian
RACE_Asian
RACE_Asian or Pacific Islander
RACE_Black or African American
RACE_Native Hawaiian or Other Pacific Islander
RACE_Not known
RACE_Other single race
RACE_Two or more races
RACE_White
MARITAL_STATUS_Divorced, widowed
MARITAL_STATUS_Never married
MARITAL_STATUS_Not known
MARITAL_STATUS_Now Married
MARITAL_STATUS_Separated
EDUCATION_1-3 years of college, university, or vocational school
EDUCATION_4 years of college, university, BA/BS, some postgraduate study, or more
EDUCATION_Grade 12 (or GED)
EDUCATION_Grades 9 to 11
EDUCATION_Less than one school grade, no schooling, nursery school, or kindergarten to Grade 8
EDUCATION_Not known
EMPLOYMENT_AT_ADMISSION_Full time
EMPLOYMENT_AT_ADMISSION_Not in labor force
EMPLOYMENT_AT_ADMISSION_Not known
EMPLOYMENT_AT_ADMISSION_Part time
EMPLOYM

In [17]:
# for model_name, model in models_to_train.items():
#     model.fit(X_train_res, y_train_res)
#     y_pred = model.predict(X_test)
#     print(f"Accuracy score for {model_name}: {accuracy_score(y_test, y_pred)}")
#     print(f"Confusion matrix: \n{confusion_matrix(y_test, y_pred)}\n")

## Tuning

In [18]:
# Define the parameter grid
param_grid = {
    'n_estimators': [500], #100, 200, 400, 
    'max_depth': [10], #5, 15, None
    'min_samples_split': [2], #2,10
    'min_samples_leaf': [2] #1,4
}

# Define model
rf = RandomForestClassifier()

# Grid
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=3)

# Fit the grid search to the data
grid_search.fit(X_train_res, y_train_res)

# Print the best parameters
print("Best parameters: ", grid_search.best_params_)

# Save the model
joblib.dump(grid_search.best_estimator_, 'best_random_forest.pkl')

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best parameters:  {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 500}


['best_random_forest.pkl']

In [19]:
%cd ..
# %cd MADS-CAPSTONE

/Users/hannah/Documents/capstone


In [20]:
%pwd

'/Users/hannah/Documents/capstone'

In [21]:
with open('best_random_forest.pkl', 'rb') as file:
    model = load(file)

y_pred = model.predict(X_test)
print(f"Accuracy score for random forest: {accuracy_score(y_test, y_pred)}")

Accuracy score for random forest: 0.7187986651835373


In [22]:
print(model.get_params())

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 10, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 2, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 500, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


## Feature Ablation

In [46]:
age_columns = ['AGE_12-14','AGE_15-17','AGE_18-20','AGE_21-24','AGE_25-29','AGE_30-34','AGE_35-39','AGE_40-44',
               'AGE_45-49','AGE_50-54','AGE_55-64','AGE_65+']

gender_columns = ['GENDER_Female','GENDER_Male','GENDER_Not known']

race_columns = ['RACE_Alaskan Native','RACE_American Indian','RACE_Asian','RACE_Asian or Pacific Islander',
                'RACE_Black or African American','RACE_Native Hawaiian or Other Pacific Islander','RACE_Not known',
                'RACE_Other single race','RACE_Two or more races','RACE_White']

marriage_columns = ['MARITAL_STATUS_Divorced, widowed','MARITAL_STATUS_Never married','MARITAL_STATUS_Not known',
                    'MARITAL_STATUS_Now Married','MARITAL_STATUS_Separated']

education_columns = ['EDUCATION_1-3 years of college, university, or vocational school',
'EDUCATION_4 years of college, university, BA/BS, some postgraduate study, or more',
'EDUCATION_Grade 12 (or GED)',
'EDUCATION_Grades 9 to 11',
'EDUCATION_Less than one school grade, no schooling, nursery school, or kindergarten to Grade 8',
'EDUCATION_Not known']

employ_columns = ['EMPLOYMENT_AT_ADMISSION_Full time','EMPLOYMENT_AT_ADMISSION_Not in labor force',
                  'EMPLOYMENT_AT_ADMISSION_Not known','EMPLOYMENT_AT_ADMISSION_Part time',
                  'EMPLOYMENT_AT_ADMISSION_Unemployed']

living_columns = ['LIVING_ARRANGEMENT_AT_ADMISSION_Dependent living','LIVING_ARRANGEMENT_AT_ADMISSION_Homeless',
                  'LIVING_ARRANGEMENT_AT_ADMISSION_Independent living','LIVING_ARRANGEMENT_AT_ADMISSION_Not known']

arrests_columns = ['ARRESTS_IN_30_DAYS_PRIOR_TO_ADMISSION_None','ARRESTS_IN_30_DAYS_PRIOR_TO_ADMISSION_Not known',
                   'ARRESTS_IN_30_DAYS_PRIOR_TO_ADMISSION_Once','ARRESTS_IN_30_DAYS_PRIOR_TO_ADMISSION_Two or more times']

referral_columns = ['PRIMARY_SOURCE_OF_REFERRAL_Alcohol/drug use care provider',
                    'PRIMARY_SOURCE_OF_REFERRAL_Court/criminal justice referral/DUI/DWI',
                    'PRIMARY_SOURCE_OF_REFERRAL_Employer/EAP', 
                    'PRIMARY_SOURCE_OF_REFERRAL_Individual (includes self-referral)',
                    'PRIMARY_SOURCE_OF_REFERRAL_Not known',
                    'PRIMARY_SOURCE_OF_REFERRAL_Other community referral',
                    'PRIMARY_SOURCE_OF_REFERRAL_Other health care provider',
                    'PRIMARY_SOURCE_OF_REFERRAL_School (educational)']

sub_columns = ['PRIMARY_SUBSTANCE_ABUSE_Alcohol','PRIMARY_SUBSTANCE_ABUSE_Barbiturates','PRIMARY_SUBSTANCE_ABUSE_Benzodiazepines',
               'PRIMARY_SUBSTANCE_ABUSE_Cocaine/crack','PRIMARY_SUBSTANCE_ABUSE_Hallucinogens','PRIMARY_SUBSTANCE_ABUSE_Heroin',
               'PRIMARY_SUBSTANCE_ABUSE_Inhalants','PRIMARY_SUBSTANCE_ABUSE_Marijuana/hashish','PRIMARY_SUBSTANCE_ABUSE_Methamphetamine/speed',
               'PRIMARY_SUBSTANCE_ABUSE_Non-prescription methadone','PRIMARY_SUBSTANCE_ABUSE_None','PRIMARY_SUBSTANCE_ABUSE_Not known',
               'PRIMARY_SUBSTANCE_ABUSE_Other amphetamines','PRIMARY_SUBSTANCE_ABUSE_Other drugs','PRIMARY_SUBSTANCE_ABUSE_Other opiates and synthetics',
               'PRIMARY_SUBSTANCE_ABUSE_Other sedatives or hypnotics','PRIMARY_SUBSTANCE_ABUSE_Other stimulants',
               'PRIMARY_SUBSTANCE_ABUSE_Other tranquilizers','PRIMARY_SUBSTANCE_ABUSE_Over-the-counter medications',
               'PRIMARY_SUBSTANCE_ABUSE_PCP']

freq_columns = ['FREQUENCY_OF_USE_Daily use','FREQUENCY_OF_USE_No use in the past month','FREQUENCY_OF_USE_Not known',
                'FREQUENCY_OF_USE_Some use']

first_use_columns = ['AGE_AT_FIRST_USE_11 years and under',
'AGE_AT_FIRST_USE_12-14 years',
'AGE_AT_FIRST_USE_15-17 years',
'AGE_AT_FIRST_USE_18-20 years',
'AGE_AT_FIRST_USE_21-24 years',
'AGE_AT_FIRST_USE_25-29 years',
'AGE_AT_FIRST_USE_30 years and older',
'AGE_AT_FIRST_USE_Not known']

alcohol_columns = ['ALCOHOL_OR_DRUG_ABUSE_Alcohol and other drugs',
'ALCOHOL_OR_DRUG_ABUSE_Alcohol only',
'ALCOHOL_OR_DRUG_ABUSE_None',
'ALCOHOL_OR_DRUG_ABUSE_Other drugs only']

dsm_columns = ['DSM_DIAGNOSIS_Alcohol abuse',
'DSM_DIAGNOSIS_Alcohol dependence',
'DSM_DIAGNOSIS_Alcohol intoxication',
'DSM_DIAGNOSIS_Alcohol-induced disorder',
'DSM_DIAGNOSIS_Anxiety disorders',
'DSM_DIAGNOSIS_Attention deficit/disruptive behavior disorders',
'DSM_DIAGNOSIS_Bipolar disorders',
'DSM_DIAGNOSIS_Cannabis abuse',
'DSM_DIAGNOSIS_Cannabis dependence',
'DSM_DIAGNOSIS_Cocaine abuse',
'DSM_DIAGNOSIS_Cocaine dependence',
'DSM_DIAGNOSIS_Depressive disorders',
'DSM_DIAGNOSIS_Not known',
'DSM_DIAGNOSIS_Opioid abuse',
'DSM_DIAGNOSIS_Opioid dependence',
'DSM_DIAGNOSIS_Other mental health condition',
'DSM_DIAGNOSIS_Other substance abuse',
'DSM_DIAGNOSIS_Other substance dependence',
'DSM_DIAGNOSIS_Schizophrenia/other psychotic disorders',
'DSM_DIAGNOSIS_Substance-induced disorder']

psych_columns = ['PSYCHIATRIC_PROBLEM_No','PSYCHIATRIC_PROBLEM_Not known','PSYCHIATRIC_PROBLEM_Yes']

insur_columns = ['HEALTH_INSURANCE_Medicaid',
'HEALTH_INSURANCE_Medicare, other (e.g. TRICARE, CHAMPUS)',
'HEALTH_INSURANCE_None',
'HEALTH_INSURANCE_Not known',
'HEALTH_INSURANCE_Private insurance, Blue Cross/Blue Shield, HMO']

pay_columns = ['PRIMARY_PAYMENT_METHOD_Medicaid',
'PRIMARY_PAYMENT_METHOD_Medicare',
'PRIMARY_PAYMENT_METHOD_No charge (free, charity, special research, teaching)',
'PRIMARY_PAYMENT_METHOD_Not known',
'PRIMARY_PAYMENT_METHOD_Other',
'PRIMARY_PAYMENT_METHOD_Other government payments',
'PRIMARY_PAYMENT_METHOD_Private insurance (Blue Cross/Blue Shield, other health insurance, workers compensation)',
'PRIMARY_PAYMENT_METHOD_Self-pay']

selfhelp_columns = ['FREQUENCY_OF_SELF_HELP_ATTENDANCE_1-3 times in the past month',
'FREQUENCY_OF_SELF_HELP_ATTENDANCE_4-7 times in the past month',
'FREQUENCY_OF_SELF_HELP_ATTENDANCE_8-30 times in the past month',
'FREQUENCY_OF_SELF_HELP_ATTENDANCE_No attendance',
'FREQUENCY_OF_SELF_HELP_ATTENDANCE_Not known',
'FREQUENCY_OF_SELF_HELP_ATTENDANCE_Some attendance, frequency is unknown']

state_columns = ['STATE_Alabama','STATE_Alaska','STATE_Arizona','STATE_Arkansas','STATE_California','STATE_Colorado',
                 'STATE_Connecticut','STATE_Delaware','STATE_District of Columbia','STATE_Florida','STATE_Georgia',
                 'STATE_Hawaii','STATE_Idaho','STATE_Illinois','STATE_Indiana','STATE_Iowa','STATE_Kansas','STATE_Kentucky',
                 'STATE_Louisiana','STATE_Maine','STATE_Maryland','STATE_Massachusetts','STATE_Michigan','STATE_Minnesota',
                 'STATE_Mississippi','STATE_Missouri','STATE_Montana','STATE_Nebraska','STATE_Nevada','STATE_New Hampshire',
                 'STATE_New Jersey','STATE_New Mexico','STATE_New York','STATE_North Carolina','STATE_North Dakota','STATE_Ohio',
                 'STATE_Oklahoma','STATE_Pennsylvania','STATE_Puerto Rico','STATE_Rhode Island','STATE_South Carolina',
                 'STATE_South Dakota','STATE_Tennessee','STATE_Texas','STATE_Utah','STATE_Vermont','STATE_Virginia',
                 'STATE_Washington','STATE_Wisconsin','STATE_Wyoming']

feature_lists = [age_columns, gender_columns, race_columns, marriage_columns, education_columns, employ_columns,
                living_columns, arrests_columns, referral_columns, sub_columns, freq_columns, first_use_columns,
                alcohol_columns, dsm_columns, psych_columns, insur_columns, pay_columns, selfhelp_columns, state_columns]

In [47]:
for x in feature_lists:
    temp_df = features_one_hot.drop(columns = x)
    
    X_train_abl, X_test_abl, y_train_abl, y_test_abl = train_test_split(temp_df, target, test_size=0.2, random_state=42)
    ros = RandomOverSampler(random_state=42)
    X_train_res_abl, y_train_res_abl = ros.fit_resample(X_train_abl, y_train_abl)
    
    # Define the parameter grid
    param_grid = {
        'n_estimators': [500], #100, 200, 400, 
        'max_depth': [10], #5, 15, None
        'min_samples_split': [2], #2,10
        'min_samples_leaf': [2] #1,4
    }

    # Define model
    rf_abl = RandomForestClassifier()

    # Grid
    grid_search_abl = GridSearchCV(estimator=rf_abl, param_grid=param_grid, cv=5, n_jobs=-1, verbose=3)

    # Fit the grid search to the data
    grid_search_abl.fit(X_train_res_abl, y_train_res_abl)

    # Save the model
    joblib.dump(grid_search_abl.best_estimator_, 'best_random_forest_abl.pkl')
    
    with open('best_random_forest_abl.pkl', 'rb') as file:
        model = load(file)

    y_pred_abl = model.predict(X_test_abl)
    
    print(f"Accuracy score for random forest removing {x}: {accuracy_score(y_test_abl, y_pred_abl)}")
        
    

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Accuracy score for random forest removing ['AGE_12-14', 'AGE_15-17', 'AGE_18-20', 'AGE_21-24', 'AGE_25-29', 'AGE_30-34', 'AGE_35-39', 'AGE_40-44', 'AGE_45-49', 'AGE_50-54', 'AGE_55-64', 'AGE_65+']: 0.7173049419990466
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Accuracy score for random forest removing ['GENDER_Female', 'GENDER_Male', 'GENDER_Not known']: 0.7157476561258541
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END max_depth=10, min_samples_leaf=2, min_samples_split=2, n_estimators=500;, score=0.728 total time= 1.5min
Accuracy score for random forest removing ['RACE_Alaskan Native', 'RACE_American Indian', 'RACE_Asian', 'RACE_Asian or Pacific Islander', 'RACE_Black or African American', 'RACE_Native Hawaiian or Other Pacific Islander', 'RACE_Not known', 'RACE_Other single race', 'RACE_Two or more races', 'RACE_White']: 0.7192436040044494
Fitting 5 folds for each of 1 candidates, total

## Feature Importances

In [48]:
#from joblib import load
#import pickle
#import numpy as np
#import matplotlib
#from matplotlib import pyplot as plt

# load model
#with open('best_random_forest.pkl', 'rb') as file:
#    model = load(file)

# extract feature importances from model
#importances = model.feature_importances_

# sort feature importances in descending order
#indices = np.argsort(importances)[::-1]

# rearrange feature names so they match the sorted feature importances
#names = [X_train.columns[i] for i in indices]


# number of features to display
#num_features = 12

# create plot
#plt.figure(figsize=(10,5))  # Adjust as needed

# create plot title
#plt.title("Feature Importance")

# add bars for top num_features
#plt.bar(range(num_features), importances[indices[:num_features]])

# add feature names as x-axis labels
#plt.xticks(range(num_features), [names[i] for i in range(num_features)], rotation=90)

# show plot
#plt.show()

## Partial Dependence Plots

In [49]:
#from sklearn.inspection import plot_partial_dependence

# number of features to display
#top_features = names[:num_features]

# define the number of rows and columns for the subplot grid
#n_rows = 6
#n_cols = 2

#fig, axs = plt.subplots(n_rows, n_cols, figsize=(18, 9))

#for i, feature in enumerate(top_features):
#    row = i // n_cols
#    col = i % n_cols
#    disp = plot_partial_dependence(model, X_train, [feature], ax=axs[row, col])
#    axs[row, col].set_title(f'PDP of {feature}')

# remove empty subplots
#if num_features < n_rows * n_cols:
#    for ax in axs.flatten()[num_features:]:
#        fig.delaxes(ax)

#plt.tight_layout()
#plt.show()